In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import pickle
with open('pivot4years.pkl','rb') as pkldfile :
    pivotByYear=pickle.load(pkldfile)

In [3]:
pivotByYear.head()

AttributeError: 'dict' object has no attribute 'head'

In [4]:
for i in pivotByYear.keys() : print(pivotByYear[i].shape) 

(1845, 1)
(1839, 1)
(1837, 1)
(1840, 1)
(1837, 1)
(1840, 1)
(1845, 1)


In [5]:
import geopandas as gpd
smallcounties = gpd.read_file('cb_2018_us_county_500k/cb_2018_us_county_500k.shp')


In [6]:
state_dict={ '25':'Massachusetts', '09':'Connecticut' , '23':'Maine', '33':'New Hampshire', '50':'Vermont' ,'44':'Rhode Island'  }
necounties=smallcounties[smallcounties['STATEFP'].isin(state_dict.keys())].copy()

In [7]:
for i in pivotByYear.keys():
    pivotByYear[i].index=pivotByYear[i].index.str.title()

In [8]:
geoOPyear=dict()
for i in pivotByYear.keys():
    geoOPyear[i]=necounties.merge(pivotByYear[i],left_on='NAME',right_on='BUYER_COUNTY')
    geoOPyear[i].drop(columns=['ALAND','AWATER'])

In [9]:
for i in geoOPyear.keys() : print(geoOPyear[i].shape) 

(67, 11)
(67, 11)
(67, 11)
(67, 11)
(67, 11)
(67, 11)
(67, 11)


In [11]:
for i in pivotByYear.keys() : print(pivotByYear[i].isna().sum()) 

TRANSACTION_DATE
2006    0
dtype: int64
TRANSACTION_DATE
2007    0
dtype: int64
TRANSACTION_DATE
2008    0
dtype: int64
TRANSACTION_DATE
2009    0
dtype: int64
TRANSACTION_DATE
2010    0
dtype: int64
TRANSACTION_DATE
2011    0
dtype: int64
TRANSACTION_DATE
2012    0
dtype: int64


In [12]:
for i in geoOPyear.keys():
    geoOPyear[i]['inMillions']=geoOPyear[i][int(i)].divide(10**6)

In [14]:
death_dtypes={'Year':int,'County':str,'Notes':str,'County Code':int,'Year Code':int , 'Deaths':pd.Int32Dtype(),'Population':int,'Crude Rate':float }
deaths=pd.read_csv('MCDNewEngland20062012.tsv',sep='\t',index_col=['County','Year'],dtype=death_dtypes, na_values=['Suppressed','Unreliable'])

In [15]:
deathspivot=deaths['Deaths'].unstack()

In [16]:
deathspivot.head()

Year,2006,2007,2008,2009,2010,2011,2012
County,,,,,,,
"Addison County, VT",NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Androscoggin County, ME",NaN,NaN,NaN,10,11,NaN,NaN
"Aroostook County, ME",NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Barnstable County, MA",22,29,22,20,20,16,19
"Belknap County, NH",NaN,12,10,NaN,NaN,NaN,NaN


In [25]:
deathspivot['CName']=deathspivot.index.str.split().str.get(0)

In [26]:
deathspivot.head()

Year,2006,2007,2008,2009,2010,2011,2012,CName
County,,,,,,,,
"Addison County, VT",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Addison
"Androscoggin County, ME",NaN,NaN,NaN,10,11,NaN,NaN,Androscoggin
"Aroostook County, ME",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aroostook
"Barnstable County, MA",22,29,22,20,20,16,19,Barnstable
"Belknap County, NH",NaN,12,10,NaN,NaN,NaN,NaN,Belknap
